In [1]:
import pandas as pd
import torch as nn
import yfinance as yf

In [48]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [17]:
from datetime import datetime

# for time stamps
yf.pdr_override()

# define the top 7 S&P 500 companies
top_7_list = ['GOOG', 'NVDA', 'AMZN', 'META', 'TSLA', 'AAPL', 'MSFT']

# set the date range
end = datetime.now()
start = datetime(2015, 4, 16)

# download the data and concatenate into a single DataFrame
top_7_dfs = [yf.download(stock, start, end).assign(Company=stock) for stock in top_7_list]
top_7_df = pd.concat(top_7_dfs)

spy_df = yf.download('SPY', start, end)
spy_df['ETF'] = 'SPY' 

bottom_20_list = ['ETSY', 'FMC', 'FRT', 'GNRC', 'HAS', 'IVZ', 'MHK', 'MKTX', 'NCLH', 'PARA',
                  'PNW', 'RHI', 'VFC', 'WHR', 'XRAY', 'ZION']

# download data and concatenate into a single DataFrame
bottom_20_dfs = [yf.download(stock, start, end).assign(Company=stock) for stock in bottom_20_list]
bottom_20_df = pd.concat(bottom_20_dfs)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [40]:
top_7_dfs[0]['Company'][0]

C:\Users\Neil\AppData\Local\Temp\ipykernel_4592\2805420630.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  top_7_dfs[0]['Company'][0]


'GOOG'

In [25]:
for l in [[spy_df], bottom_20_dfs, top_7_dfs]:
    for dataset in l:
        dataset['Returns'] = (1 - dataset['Close']/dataset['Open']) * 100

In [88]:
spy_main = pd.DataFrame()
spy_temp = spy_df[['Volume', 'Returns']]
spy_columns = spy_temp.rename(columns = {'Volume': 'SPY_Volume', 'Returns': 'SPY_Returns'})
for dataset in bottom_20_dfs:
    temp_df = pd.concat([dataset, spy_columns], axis=1)
    spy_main = pd.concat([spy_main, temp_df])
spy_main

,Open,High,Low,Close,Adj Close,Volume,Company,Returns,SPY_Volume,SPY_Returns
Date,,,,,,,,,,
2015-04-16,31.000000,35.740002,28.219999,30.000000,30.000000,19763300,ETSY,3.225806,68934900,-0.161880
2015-04-17,29.770000,30.299999,26.510000,27.580000,27.580000,3965500,ETSY,7.356401,191113200,0.473823
2015-04-20,28.770000,28.900000,24.870001,24.900000,24.900000,3076200,ETSY,13.451515,92189500,-0.377886
2015-04-21,24.969999,26.040001,24.559999,25.750000,25.750000,2184700,ETSY,-3.123751,72559800,0.507900
2015-04-22,26.000000,26.240000,24.950001,25.120001,25.120001,1442500,ETSY,3.384612,78264600,-0.295229
...,...,...,...,...,...,...,...,...,...,...
2024-03-11,42.270000,42.439999,41.240002,41.610001,41.610001,2607800,ZION,1.561391,62557200,-0.156713
2024-03-12,41.630001,42.040001,40.669998,40.900002,40.900002,2254900,ZION,1.753542,73114400,-0.648557
2024-03-13,40.650002,41.369999,40.570000,41.070000,41.070000,1956600,ZION,-1.033206,55104100,0.220459


In [89]:
mag7_main_T = pd.DataFrame()
temp_main = pd.DataFrame()

for company in top_7_dfs:
    company_temp = company[['Volume', 'Returns']]
    spy_columns = company_temp.rename(columns = {'Volume': f"{str(company['Company'][0])}" + '_Volume', 'Returns': f"{str(company['Company'][0])}" + '_Returns'})
    for dataset in bottom_20_dfs:
        temp_df = pd.concat([dataset, spy_columns], axis=1)
        temp_main = pd.concat([temp_main, temp_df])
    mag7_main_T = pd.concat([mag7_main_T, temp_main], axis=1)
    temp_main = pd.DataFrame()

mag7_main = mag7_main_T.T
mag7_main = mag7_main.drop_duplicates().T
mag7_main

,Open,High,Low,Close,Adj Close,Volume,Company,Returns,GOOG_Volume,GOOG_Returns,...,AMZN_Volume,AMZN_Returns,META_Volume,META_Returns,TSLA_Volume,TSLA_Returns,AAPL_Volume,AAPL_Returns,MSFT_Volume,MSFT_Returns
Date,,,,,,,,,,,,,,,,,,,,,
2015-04-16,31.0,35.740002,28.219999,30.0,30.0,19763300,ETSY,3.225806,25997180,-0.735986,...,41608000,-0.612475,13769700,0.194014,24886500,0.48147,113476000,0.087108,22509700,-0.500594
2015-04-17,29.77,30.299999,26.51,27.58,27.58,3965500,ETSY,7.356401,43037837,0.872012,...,76794000,1.84774,24215000,0.859112,37048500,-0.878091,207828000,0.637199,42387600,0.119989
2015-04-20,28.77,28.9,24.870001,24.9,24.9,3076200,ETSY,13.451515,33585958,-1.860731,...,100322000,-2.89526,28796800,-1.900902,38389500,0.73024,188217200,-1.616627,46057700,-2.827703
2015-04-21,24.969999,26.040001,24.559999,25.75,25.75,2184700,ETSY,-3.123751,36895018,0.658586,...,92870000,0.033223,27171900,0.452378,51487500,-1.754128,129740400,0.928964,26013800,0.837211
2015-04-22,26.0,26.24,24.950001,25.120001,25.120001,1442500,ETSY,3.384612,31871263,-0.929078,...,69494000,0.538395,45548000,-0.367644,117945000,-3.265874,150618000,-1.283563,25064300,-0.74995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-11,42.27,42.439999,41.240002,41.610001,41.610001,2607800,ZION,1.561391,22536400,-1.364263,...,28484800,1.348168,20428300,2.70015,85391500,-1.322318,60139500,0.109866,16120800,-0.188225
2024-03-12,41.630001,42.040001,40.669998,40.900002,40.900002,2254900,ZION,1.753542,19019700,-0.990955,...,36610600,-1.089337,15448200,-1.315734,87391700,0.129387,59825400,-0.046204,22457000,-1.879202
2024-03-13,40.650002,41.369999,40.57,41.07,41.07,1956600,ZION,-1.033206,19637000,-0.50693,...,30772600,-0.375215,12090700,-0.036334,106524500,2.062992,52488700,0.949238,17115900,0.717532


In [90]:
spy_main = pd.get_dummies(spy_main, columns=['Company'])
mag7_main = pd.get_dummies(mag7_main, columns=['Company'])

# SPY MODEL

In [92]:
def create_chunks(dataframe):
    X_chunks = []
    Y_chunks = []
    
    num_rows = len(dataframe)
    start = 0
    
    # List of company columns
    company_columns = [col for col in dataframe.columns if col.startswith('Company_')]
    
    while start < num_rows - 2:
        if (dataframe.index[start + 1] - dataframe.index[start]).days == 1 and \
           (dataframe.index[start + 2] - dataframe.index[start + 1]).days == 1:
            
            # Check if the same company column is True across all three days
            if dataframe.iloc[start:start+3][company_columns].sum(axis=0).max() == 3:
                # The chunk pertains to the same company
                X_chunk = dataframe.iloc[start:start+2, :]
                Y_chunk = dataframe.iloc[start + 2]['Returns']
                
                X_chunks.append(X_chunk)
                Y_chunks.append(Y_chunk)
                
                start += 1
            else:
                # The chunk does not pertain to the same company, skip to the next possible start
                start += 1
        else:
            # Dates are not consecutive, skip to the next possible start
            start += 1
    
    return X_chunks, Y_chunks

X_spy, y_spy = create_chunks(spy_main)

In [98]:
X_spy[0]

,Open,High,Low,Close,Adj Close,Volume,Returns,SPY_Volume,SPY_Returns,Company_ETSY,...,Company_MHK,Company_MKTX,Company_NCLH,Company_PARA,Company_PNW,Company_RHI,Company_VFC,Company_WHR,Company_XRAY,Company_ZION
Date,,,,,,,,,,,,,,,,,,,,,
2015-04-20,28.770000,28.900000,24.870001,24.90,24.90,3076200,13.451515,92189500,-0.377886,True,...,False,False,False,False,False,False,False,False,False,False
2015-04-21,24.969999,26.040001,24.559999,25.75,25.75,2184700,-3.123751,72559800,0.507900,True,...,False,False,False,False,False,False,False,False,False,False


In [99]:
y_spy[0]

3.3846121567946197

In [282]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import models
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler
import time

In [172]:
X_train, X_test, y_train, y_test = train_test_split(X_spy, y_spy, test_size=0.2, random_state=42)

In [204]:
def PreProcess(X_data):
    processed_arrays = []
    for X in X_data:
        X_arr = X.to_numpy(dtype=np.float32)
        X_reshaped = X_arr.reshape(1, 1, 2, X.shape[1])
        processed_arrays.append(X_reshaped)
    concatenated_array = np.concatenate(processed_arrays, axis=0)
    processed_tensor = torch.tensor(concatenated_array, dtype=torch.float32)
    
    return processed_tensor

In [205]:
X_spy_train = PreProcess(X_train)
X_spy_test = PreProcess(X_test)
X_spy_train.shape

torch.Size([16524, 1, 2, 25])

In [206]:
X_spy_train[0]

tensor([[[ 2.0307e+02,  2.1192e+02,  1.9100e+02,  2.1075e+02,  2.1075e+02,
           3.6461e+06, -3.7819e+00,  1.0728e+08, -7.3831e-01,  1.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 2.0999e+02,  2.1288e+02,  2.0226e+02,  2.0910e+02,  2.0910e+02,
           2.1790e+06,  4.2383e-01,  7.2434e+07, -1.4081e+00,  1.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]]])

In [207]:
y_spy_train = torch.tensor([y_train], dtype=torch.float32)
y_spy_train = y_spy_train.view(-1)
y_spy_test = torch.tensor([y_test], dtype=torch.float32)
y_spy_test = y_spy_test.view(-1)

y_spy_train.shape

torch.Size([16524])

In [233]:
# Hyperparameters

num_classes = 1
num_epochs = 100
batch_size = 10
learning_rate = 0.0001

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [209]:
train_dataset = TensorDataset(X_spy_train, y_spy_train)
test_dataset = TensorDataset(X_spy_test, y_spy_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [210]:
ex = iter(train_loader)
next(ex)[0]

tensor([[[[ 4.3120e+01,  4.3350e+01,  4.2870e+01,  4.2960e+01,  3.5095e+01,
            2.5081e+06,  3.7106e-01,  5.6906e+07,  5.0768e-01,  0.0000e+00,
            0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00],
          [ 4.3030e+01,  4.3230e+01,  4.2250e+01,  4.2330e+01,  3.4580e+01,
            2.1235e+06,  1.6268e+00,  5.5978e+07,  2.0948e-01,  0.0000e+00,
            0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]]],


        [[[ 8.8840e+01,  8.8840e+01,  8.7290e+01,  8.7990e+01,  7.4890e+01,
            1.6637e+06,  9.5677e-01,  7.5768e+07,  9.3772e-01,  0.0000e+00,
            0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
      

In [286]:
class MyCNN(nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(2, 5), stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(2, 3), stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(1, 3), stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2))
        self.dropout1 = nn.Dropout(0.3)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(1, 3), stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(1, 3), stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.dropout2 = nn.Dropout(0.4)

        # Fully connected layers
        # Adjusted for additional conv layers. Dynamically calculate in_features in forward().
        self.fc1 = nn.Linear(in_features=512, out_features=512)  # Placeholder, will adjust dynamically
        self.fc2 = nn.Linear(in_features=512, out_features=128)
        self.dropout3 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(in_features=128, out_features=1)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.dropout1(x)
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = F.relu(self.bn5(self.conv5(x)))
        x = self.dropout2(x)
        
        # Before flattening, dynamically adjust fc1's in_features based on the conv5 output
        n_size = x.size()[1] * x.size()[2] * x.size()[3]
        self.fc1 = nn.Linear(in_features=n_size, out_features=512).to(x.device)
        
        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.dropout3(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [287]:
model = MyCNN().to(device)

def custom_sign_loss(output, target):
    sign_agreement = ((output * target) >= 0).float()  # 1 when signs agree, 0 otherwise
    magnitude_loss = torch.abs(output - target)  # or any other magnitude-based loss
    return torch.mean((1 - sign_agreement) * magnitude_loss)

def custom_hinge_loss(outputs, targets):
    targets = targets.view_as(outputs)
    loss = torch.where(targets * outputs < 0, torch.abs(outputs), torch.tensor(0.0, device=outputs.device))
    return torch.mean(loss)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

scheduler = ReduceLROnPlateau(optimizer, 'min')

In [288]:
loss_min = np.inf

for epoch in range(num_epochs):
    start_time = time.time()
    model.train()  # Set model to training mode
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        
        # Compute loss
        loss = criterion(outputs, targets.view(-1, 1))
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Print statistics
        running_loss += loss.item()
    
    loss_train = running_loss / len(train_loader)
    
    # Validation loop
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_loss = 0.0
        correct_signs = 0  # Counter for predictions with the correct sign
        total_samples = 0
        
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            
            # Calculate loss
            loss = criterion(outputs, targets.view(-1, 1))
            val_loss += loss.item() * inputs.size(0)  # Multiply by batch size
            
            # Calculate the accuracy based on the sign
            predicted_signs = torch.sign(outputs)
            target_signs = torch.sign(targets.view(-1, 1))
            correct_signs += torch.sum(predicted_signs == target_signs).item()
            if correct_signs > 100 and epoch > 25:
                print(targets, outputs)
            total_samples += inputs.size(0)
        
        val_loss /= len(test_loader.dataset)
        val_acc = correct_signs / total_samples
    
    epoch_end_time = time.time()  # End time of the epoch
    epoch_duration = epoch_end_time - start_time  # Calculate duration of the epoch

    scheduler.step(val_loss)

    for param_group in optimizer.param_groups:
        if epoch%10 == 0:
            print("Current learning rate is: {}".format(param_group['lr']))

    print(f"Epoch {epoch+1}/{num_epochs} - Training Loss: {loss_train:.4f}, Validation Loss: {val_loss:.4f}, Val_Accuracy: {val_acc:.4f}, Time: {epoch_duration:.2f} sec")

    if val_loss < loss_min:
        loss_min = val_loss
        torch.save(model.state_dict(), './CNN_SPY_Model.pth')
        print(f'\n Minimum Val Loss of {val_loss:.4f} at epoch {epoch+1}')
        print(f'Model Saved \n')

Current learning rate is: 0.001
Epoch 1/100 - Training Loss: 4.2906, Validation Loss: 4.2092, Val_Accuracy: 0.4954, Time: 43.51 sec
Epoch 2/100 - Training Loss: 4.2794, Validation Loss: 4.2305, Val_Accuracy: 0.4993, Time: 43.66 sec
Epoch 3/100 - Training Loss: 4.2770, Validation Loss: 4.2092, Val_Accuracy: 0.4978, Time: 43.92 sec
Epoch 4/100 - Training Loss: 4.2745, Validation Loss: 4.2045, Val_Accuracy: 0.4969, Time: 45.92 sec
Epoch 5/100 - Training Loss: 4.2761, Validation Loss: 4.2074, Val_Accuracy: 0.4971, Time: 44.27 sec
Epoch 6/100 - Training Loss: 4.2756, Validation Loss: 4.2090, Val_Accuracy: 0.5041, Time: 44.42 sec


KeyboardInterrupt: 

# Mag 7 Model

In [293]:
X_mag, y_mag = create_chunks(mag7_main)

In [297]:
X_train, X_test, y_train, y_test = train_test_split(X_mag, y_mag, test_size=0.2, random_state=42)

X_spy_train = PreProcess(X_train)
X_spy_test = PreProcess(X_test)

y_spy_train = torch.tensor([y_train], dtype=torch.float32)
y_spy_train = y_spy_train.view(-1)
y_spy_test = torch.tensor([y_test], dtype=torch.float32)
y_spy_test = y_spy_test.view(-1)

train_dataset = TensorDataset(X_spy_train, y_spy_train)
test_dataset = TensorDataset(X_spy_test, y_spy_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

X_spy_train.shape

torch.Size([16524, 1, 2, 37])

In [296]:
loss_min = np.inf

for epoch in range(num_epochs):
    start_time = time.time()
    model.train()  # Set model to training mode
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        
        # Compute loss
        loss = criterion(outputs, targets.view(-1, 1))
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Print statistics
        running_loss += loss.item()
    
    loss_train = running_loss / len(train_loader)
    
    # Validation loop
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_loss = 0.0
        correct_signs = 0  # Counter for predictions with the correct sign
        total_samples = 0
        
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            
            # Calculate loss
            loss = criterion(outputs, targets.view(-1, 1))
            val_loss += loss.item() * inputs.size(0)  # Multiply by batch size
            
            # Calculate the accuracy based on the sign
            predicted_signs = torch.sign(outputs)
            target_signs = torch.sign(targets.view(-1, 1))
            correct_signs += torch.sum(predicted_signs == target_signs).item()
            if correct_signs > 100 and epoch > 25:
                print(targets, outputs)
            total_samples += inputs.size(0)
        
        val_loss /= len(test_loader.dataset)
        val_acc = correct_signs / total_samples
    
    epoch_end_time = time.time()  # End time of the epoch
    epoch_duration = epoch_end_time - start_time  # Calculate duration of the epoch

    scheduler.step(val_loss)

    for param_group in optimizer.param_groups:
        if epoch%10 == 0:
            print("Current learning rate is: {}".format(param_group['lr']))

    print(f"Epoch {epoch+1}/{num_epochs} - Training Loss: {loss_train:.4f}, Validation Loss: {val_loss:.4f}, Val_Accuracy: {val_acc:.4f}, Time: {epoch_duration:.2f} sec")

    if val_loss < loss_min:
        loss_min = val_loss
        torch.save(model.state_dict(), './CNN_Mag7_Model.pth')
        print(f'\n Minimum Val Loss of {val_loss:.4f} at epoch {epoch+1}')
        print(f'Model Saved \n')

Current learning rate is: 0.001
Epoch 1/100 - Training Loss: 4.2751, Validation Loss: 4.2087, Val_Accuracy: 0.4923, Time: 120.11 sec

 Minimum Val Loss of 4.2087 at epoch 1
Model Saved 



KeyboardInterrupt: 